<a href="https://colab.research.google.com/github/Zen-Sherbert/Proteus-Attention/blob/main/TinyPlayground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Clone your GitHub repository
!git clone https://github.com/Zen-Sherbert/Proteus-Attention.git

# 2. Change the working directory to the cloned folder
%cd Proteus-Attention

# 3. (Optional) List the files to show the user everything is there
print("Repository files:")
!ls -F

Cloning into 'Proteus-Attention'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 127 (delta 39), reused 92 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (127/127), 150.01 KiB | 6.82 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/Proteus-Attention
Repository files:
examples/  MANIFEST.in	   scripts/  tests/
LICENSE    pyproject.toml  src/      tinytoy_run.txt


In [8]:
# Install the proteus_attention package
!pip install .

Processing /content/Proteus-Attention
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for proteus-attention: filename=proteus_attention-0.1.0-py3-none-any.whl size=74525 sha256=6d29034ed42007fb9a820cb698ab41b4d1f26ef1803b454f86ae35c423feee4a
  Stored in directory: /root/.cache/pip/wheels/e1/38/b5/a9e731a0381ba75a59307d7a0512b8a0ac1e70c603ff48dc4f
Successfully built proteus-attention


In [6]:
# 4. Run the tiny toy script on the GPU
!python src/proteus_attention/kernels/tinytoy.py

[Config] SDPA fast-path disabled; benchmarking Flux kernels only.
--- Starting Attention Benchmark ---
Device: cuda
Sequence lengths: [128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072, 262144, 524288, 1048576]

[Probe] Standard testing seq_len=128 batch=4 runs=3
[Probe] Standard testing seq_len=256 batch=4 runs=3
[Probe] Standard testing seq_len=512 batch=4 runs=3
[Probe] Standard testing seq_len=1024 batch=4 runs=3
[Probe] Standard testing seq_len=2048 batch=4 runs=3
[Probe] Standard testing seq_len=4096 batch=4 runs=3
[Probe] Standard testing seq_len=8192 batch=1 runs=1
[Probe] Standard testing seq_len=16384 batch=1 runs=1
[Probe] Standard testing seq_len=32768 batch=1 runs=1
[Probe] Standard failed at sequence length 32768: CUDA out of memory. Tried to allocate 16.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 14.44 GiB is free. Process 42344 has 308.00 MiB memory in use. Of the allocated memory 138.13 MiB is allocated by PyTorch, and 43.87 MiB is reserve

In [9]:
# Run the chunked_flux_smoke.py script
!python scripts/chunked_flux_smoke.py

Chunked Flux: 100% 16/16 [00:02<00:00,  7.45chunk/s]

=== Chunked Flux Smoke Test ===
device              : cuda
sequence length     : 1000000
retained tokens     : 32768 (3.28%)
chunks processed    : 16
stream latency (ms) : 2149.73
stream throughput   : 465,175.64 tok/s
final latency (ms)  : 110.69
final throughput    : 296,036.17 tok/s
final peak memory   : 1257.5 MB
overall throughput  : 427,471.08 tok/s



In [10]:
# Run the chunked_flux_tests.py script
!python scripts/chunked_flux_tests.py

config: seq_len=8,192 chunk_len=1,024 buffer_tokens=8 per_chunk_budget=2 (keep ratio 1.000)
=== Needle recall (high router score) ===
retained sentinel? True | keep_indices=[1047, 2891, 3937, 4357, 4369, 5231, 7174, 7233]

=== DNA teleportation hypothesis ===
without DNA retained? False | with DNA retained? True

=== Ordering sanity (RoPE alignment) ===
keep_indices monotonic? True

=== Needle cluster stress test ===
[cluster] needles kept 1/10 (recall=0.10) with per_chunk_budget=1
[cluster-adapt] needles kept 8/10 (recall=0.80) margin=0.1 max_extra=8

=== Fading signal sweep ===
[fading] sensitivity sweep:
  boost=5.00 -> recall 100.00% (20/20)
  boost=2.00 -> recall 100.00% (20/20)
  boost=1.00 -> recall 100.00% (20/20)
  boost=0.50 -> recall 100.00% (20/20)
  boost=0.25 -> recall 55.00% (11/20)
  boost=0.10 -> recall 0.00% (0/20)

=== Jigsaw puzzle synthesis ===
[jigsaw] both clues retained? True | keep_indices=[512, 1287, 2313, 2641, 4823, 5179, 6712, 7851]

=== Sample text demo ==

In [11]:
# Run the run_fluxchunk_sweep.py script
!python scripts/run_fluxchunk_sweep.py

[FluxChunk] seq_len=1,000,000 buffer=1,000 alpha=1.000 device=cuda
  chunk: 1038.4 ms | 962,974 tok/s
  final: 2917.0 ms | 343 tok/s
  peak VRAM: 113.4 MB
  total: 3971.0 ms | 251,825 tok/s
  retained=1,000 (0.0010) | fallback=False
[FluxChunk] seq_len=2,000,000 buffer=10,000 alpha=1.000 device=cuda
  chunk: 2497.4 ms | 800,828 tok/s
  final: 17.3 ms | 578,983 tok/s
  peak VRAM: 225.7 MB
  total: 2525.4 ms | 791,942 tok/s
  retained=10,000 (0.0050) | fallback=False
[FluxChunk] seq_len=3,000,000 buffer=30,000 alpha=1.000 device=cuda
  chunk: 2860.2 ms | 1,048,866 tok/s
  final: 29.5 ms | 1,018,330 tok/s
  peak VRAM: 372.9 MB
  total: 2909.5 ms | 1,031,101 tok/s
  retained=30,000 (0.0100) | fallback=False
[FluxChunk] seq_len=4,000,000 buffer=80,000 alpha=1.000 device=cuda
  chunk: 4061.8 ms | 984,789 tok/s
  final: 71.3 ms | 1,121,594 tok/s
  peak VRAM: 694.3 MB
  total: 4190.2 ms | 954,599 tok/s
  retained=80,000 (0.0200) | fallback=False
[FluxChunk] seq_len=5,000,000 buffer=100,000 alp